In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('data/individual_calibration_set.csv')
data.head()

,Unnamed: 0,B_x,B_y,B_z,M_1,M_2,M_3
0,0,-53.033009,0.0,-17.677670,-10.00,0.0,0.0
1,1,-52.820877,0.0,-17.606959,-9.96,0.0,0.0
2,2,-52.608745,0.0,-17.536248,-9.92,0.0,0.0
3,3,-52.396612,0.0,-17.465537,-9.88,0.0,0.0
4,4,-52.184480,0.0,-17.394827,-9.84,0.0,0.0
